# Seagrass Analysis

The following Code consists of 6 sections: Preparation, Cropping, Reflection Removal, Seagrass Detection, Selection Refining, and Seagrass Area Calculation.  
The goal is to identify all the seagrass in a given tif file to then calculate its area in pixels, square metres, and hectars.  
Further, it exports the detected seagrass areas as a shapefile which can be used in QGIS.   
In the beginning of every section, the input and final output of the section is mentioned, and a brief explanation of the code is given. 

## Preparation

Overview of the different libraries and their functions:  
CV2: Short for Computer Vision, Image processing library  
Numpy and Pandas: General data processing libraries  
Ctypes: used to access another programming langauge called C++, used for accessing files within a computer (xxx)  
CSV: Short for Comma Seperated Values, used to save the coordinates in a useful format  
Shapely: for geometric shapes, used to extract the area of interest  
tqdm: used to display progress bars for parts of the code that have significant running times  
os: useful for saving iamges in specified folders by joining paths to specific images

In [2]:
#Importing the necessary libraries
import cv2
import numpy as np
import csv
import os
from shapely import Polygon
from tqdm import tqdm

In [10]:
# defining the source path, results path, and image name
# saving the image name as a variable allows for renaming and clear storage in the final section of the code
# saves the image in a variable 
source_path = r"C:\Users\aline\Seagrass Mapping Drone\Tifffiles"
image_name = "AK_tryout"
image = cv2.imread(os.path.join(source_path, f"{image_name}.tif"))
results_path = r"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs"

if image is None:
    print("Error: Image not loaded correctly.")

## Part 1: Cropping the tif to the area of interest

*Input: original tif (image)  
Output: cropped image (cropped_image)*  
This section of the code allows the user to select coordinates around the area of interest (the sea) and then uses a mask  to crop out everything ouside of that area. The mask is created by using the selected coordiantes as the corner points of a polygon. The mask and the original picture are then overlayed, and only the area within the polygon remains in the output image. 

In [14]:
#get the coordinates of the area of interest (crops out the land area and keeps the sea)
def main():
 
    # Open a text file to save the coordinates
    with open("coordinates_shoreline.csv", "w") as f:
            
        # Display instructions in the console
        print("Select a series of points by clicking on the image.")
        print("Press 'Enter' to finish the selection.")
        
        # Create a window for displaying the image
        cv2.namedWindow("Point Selection")
        cv2.imshow("Point Selection", image)
        
        # List to store the red points
        red_points = []
        
        # Wait for the user to select points by clicking
        def on_mouse_click(event, x, y, flags, param):
            if event == cv2.EVENT_LBUTTONDOWN:
     
                # Add the point to the list of red points
                red_points.append((x, y))
                
                # Display the clicked point in red
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
                
                # Print the coordinates
                print("Coordinates:", (x, y))
        
                # Save the coordinates in the csv file
                with open("coordinates_shoreline.csv", mode="w", newline="") as file:
                    writer = csv.writer(file)
                    writer.writerows(red_points)
                
                # Draw a green line between the last two red points
                if len(red_points) >= 2:
                    pt1 = red_points[-2]
                    pt2 = red_points[-1]
                    cv2.line(image, (pt1[0], pt1[1]), (pt2[0], pt2[1]), (0, 0, 255), 2)
                
                # Update the image display
                cv2.imshow("Point Selection", image)

        cv2.setMouseCallback("Point Selection", on_mouse_click)
        
        # Wait for the user to press the 'Enter' key to finish the selection
        while True:
            key = cv2.waitKey(0)
            if key == 13:  # 'Enter' key
                break
        
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Select a series of points by clicking on the image.
Press 'Enter' to finish the selection.
Coordinates: (3158, 7601)
Coordinates: (3146, 7152)
Coordinates: (2547, 7105)
Coordinates: (2709, 7075)
Coordinates: (2658, 6463)
Coordinates: (2183, 6104)
Coordinates: (1781, 5891)
Coordinates: (1859, 5403)
Coordinates: (2850, 4138)
Coordinates: (2970, 3922)
Coordinates: (2854, 3595)
Coordinates: (2839, 3254)
Coordinates: (3011, 2584)
Coordinates: (3133, 2474)
Coordinates: (3082, 2301)
Coordinates: (3178, 1907)
Coordinates: (3371, 1890)
Coordinates: (3413, 1618)
Coordinates: (3823, 1385)
Coordinates: (4492, 1150)
Coordinates: (5073, 1039)
Coordinates: (5240, 782)
Coordinates: (5494, 476)
Coordinates: (5605, 244)
Coordinates: (6023, 116)
Coordinates: (6148, 76)
Coordinates: (6546, 66)
Coordinates: (7209, 290)
Coordinates: (7327, 341)
Coordinates: (7339, 534)
Coordinates: (8021, 617)
Coordinates: (8502, 782)
Coordinates: (9556, 1175)
Coordinates: (9774, 1278)
Coordinates: (9260, 1888)
Coordinates:

In [15]:
#creating a mask based on the selected coordinates

# saving the coordinates in a list format
with open("coordinates_shoreline.csv", "r") as file:
    coord = list(csv.reader(file, quoting=csv.QUOTE_NONNUMERIC))

# read the image into cv2 and create an array of zeros in the same shape as the original image  
canvas = np.zeros((np.shape(image)), dtype = 'uint8')

# convert the coordinates to an array of the type int32
coord_array = np.array(coord, np.int32)

# feed the coordinate array into a new variable called pts (points) which is in a specific shape
# draw polylines in shape of the sea area on the canvas, call it mask (the number 255 3x means it will be white)
# this command only draws the lines but has no fill, use fillConvexPoly to fill the polygon
pts = np.reshape(coord_array, (-1, 1, 2))
mask = cv2.polylines(canvas, pts, True, (255,255,255))
cv2.fillConvexPoly(mask, pts, (255, 255, 255))

# display the created mask in a window (optional)
cv2.namedWindow("Mask", cv2.WINDOW_NORMAL)
cv2.imshow('Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
# saving the mask as a new image and displaying it 
# bitwise_and overlaps the two images and keeps only the bits (pixels) where both of them overlap

cropped_image = cv2.bitwise_and(mask, image)
cv2.imshow('Cropped Image', cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Part 2: Removing the Reflection

*input: cropped image (cropped_image)  
output: image without reflection (image_without_reflection)*  
This section of the code detects the reflection in the area of interest and then removes it. First, the image is converted from BGR to HSV (blue green red to hue saturation value), of which we are interested in the saturation specifically, as it is very high in when there is reflection. A modifiable threshold is given: all the pixels with a value within that threshold (meaning all pixels lighter than the minimum value) are then marked as reflections. The area around the relfections is then used to get the mean BGR colors, which are in turn applied to the reflection. A tif file in which the selected pixels are highlighted is created to allow for user interaction and easier adatpation of the code, and a tif file without the reflection is the final result of this section. 

In [22]:
# convert the BGR image to HSV and split to access the saturation only
imghsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
(h, s, v) = cv2.split(imghsv)

# create a binary image based on the saturation: the first value can be modified, the second (255) should remain the same
# Increasing the first number will increase the number of reflections detected, decreasing it will decrease the number of reflections detected
_, binary = cv2.threshold(s, 25, 255, cv2.THRESH_BINARY_INV)
cropped_binary = cv2.bitwise_and(binary, mask[:,:,0])
reflection_vs_original = np.concatenate((cropped_image[:,:,0], cropped_binary), axis=0)
cv2.imshow('Detected Reflection', reflection_vs_original)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
#define function that replaces white pixels with average colour around them
            
def replace_white_color(image, contours): 
    i = 0
    for contour in contours: 
        #Create a blank mask for each contour
        white_mask = np.zeros(cropped_image.shape[:2], dtype=np.uint8)
        
        # Draw contour onto the mask
        cv2.drawContours(white_mask, [contour], -1, 255, thickness=cv2.FILLED)

        # Dilate the mask to include a border around the contour
        kernel = np.ones((5, 5), np.uint8)
        dilated_white_mask = cv2.dilate(white_mask, kernel, iterations=1)

        # Calculate the area covered by the dilated mask excluding the original mask
        mask_diff = cv2.subtract(dilated_white_mask, white_mask)

        # Apply the mask difference to the original image and calculate the mean color
        mean_color = cv2.mean(image, mask=mask_diff)[:3]            
        mean_color = tuple([int(c) for c in mean_color])
    
        # Replace the color in the modified image only where white_mask is 255
        image[dilated_white_mask == 255] = mean_color
        i += 1
        pbar.update(1)
    print("Number of detected and processed contours:", len(contours))
    
    return image

In [24]:
# Find all the contours (white zones) in the binary image
contours, _ = cv2.findContours(cropped_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on the original image and run the reflection removal code
image_with_contours = cropped_image.copy()
cv2.drawContours(image_with_contours, contours, -1, (0, 255, 255), -1)
print(len(contours), 'contours were detected. Based on previous experience removing the reflection will take approximately', round(len(contours)/1000*6.7, 0), 'minutes.')

8685 contours were detected. Based on previous experience removing the reflection will take approximately 58.0 minutes.


In [25]:
# Remove the reflection by replacing white contours with the average color of the area around them
# displays a progress bar
# if you run this code multiple times, there might be an issue with the progress bar (for every perecentage a new line appears) 
# if this happens, run: pbar.close() before the rest of the code
#pbar.close()
pbar = tqdm(total = len(contours), desc = 'Processed Contours', colour = 'cyan')
image_filled = replace_white_color(cropped_image.copy(), contours)
pbar.close()

Processed Contours: 100%|██████████████████████████████████████████████████████████| 8685/8685 [50:56<00:00,  2.84it/s]

Number of detected and processed contours: 8685


In [26]:
# Display the image with the contours of white pixels highlighted in yellow
concatenated_image = np.concatenate((image_with_contours, image_filled), axis=0) 

cv2.imshow("Detected Reflection (yellow) and Image without Reflection", concatenated_image)
cv2.imwrite(os.path.join(results_path, f"{image_name}_withoutReflection.tif"), image_filled)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Part 3: Identifying the Seagrass

*input: cropped image without reflection (image_without_reflection)  
output: binary image where seagrass appears in white (dbo_binary)*  
This section of the code highlights all the seagrass within a given tif file. First, the image is converted from RGB (red green blue) to HSV (hue saturation value) as these are closer to human perception of color and thus easier to work with. The threshold HSV values can be selected, based on which the code selects all areas that fall within the threshold. The code then opens a window in which an outline of all the selected seagrass areas is shown.  
As the values between different picture, the code allows the user to select points on the outlined image, for which the HSV values are returned. This helps the user adapt the threshold.

In [27]:
image_without_reflection = cv2.imread(os.path.join(results_path, f"{image_name}_withoutReflection.tif"), image_filled)

def segment_dark_blue_objects(image, lower_bound, upper_bound):
    hsv_image = cv2.cvtColor(image_without_reflection, cv2.COLOR_BGR2HSV)
    dark_blue_mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    dark_blue_objects = cv2.bitwise_and(image, image, mask=dark_blue_mask)
    return dark_blue_objects
    
def draw_outlines(binary_image):
    # find contours of binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # draw contours
    image_with_contours = image_without_reflection.copy()
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 255), 2)

    # display outlines
    cv2.imshow('Outlines', image_with_contours)
    cv2.imwrite(os.path.join(results_path, f'{image_name}_outlines.tif'), image_with_contours)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [28]:
# Segment dark blue objects
blue_min = np.array([92, 0, 70])
blue_max = np.array([110, 190, 200])
dark_blue_objects = segment_dark_blue_objects(image_without_reflection, blue_min, blue_max)

# create binary of the dark blue objects (dbo) for further processing
dbo_grayscale = cv2.cvtColor(dark_blue_objects, cv2.COLOR_RGB2GRAY)
_, dbo_binary = cv2.threshold(dbo_grayscale, 1, 255, cv2.THRESH_BINARY)

draw_outlines(dbo_binary)

In [114]:
image_with_contours = cv2.imread(os.path.join(results_path, f'{image_name}_outlines.tif'))
cv2.namedWindow("Seagrass Point Selection")
cv2.imshow("Seagrass Point Selection", image_with_contours)
seagrass_points = []
def on_mouse_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Add the point to the list of red points
        seagrass_points.append((y, x))
                
        # Display the clicked point in red
        cv2.circle(image, (y, x), 2, (0, 255, 0), -1)

cv2.setMouseCallback("Seagrass Point Selection", on_mouse_click)
cv2.waitKey(0)
cv2.destroyAllWindows()

seagrass_points
HSV = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
for point in seagrass_points:
    print(HSV[point])

[ 91 138 155]
[ 90 126 152]
[ 90 119 154]
[ 90 141 137]
[ 91 130 145]
[ 94 127 130]
[ 92  95 179]
[ 91 114 148]


## Part 4: Refining Selection

In [100]:
# Define the functions to remove noise and fill seagrass patches
def remove_noise(image, kernel_size, iterations):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
    image_without_noise = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations = iterations)
    return image_without_noise
    

def fill_holes(image, kernel_size, iterations):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
    image_with_filled_patches = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel, iterations=iterations)
    return image_with_filled_patches

In [116]:
# Remove noise and fill up the holes in the seagrass patches
# the kernel size and amount of iterations can be changed

image_without_noise = remove_noise(dbo_binary, 3, 5)
image_with_filled_patches = fill_holes(image_without_noise, 3, 10)

draw_outlines(image_with_filled_patches)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [117]:
# when happy with the result, save the refined (binary) image to the results folder

cv2.imwrite(os.path.join(results_path, f"{image_name}_SeagrassBinary.tif"), image_with_filled_patches)

True

Congratulations, this part of the code is now done! To continue working with the seagrass areas, please refer to the protocol and move on to section 2, in which the binary tif is georeferenced and transformed into a vector shapefile that can be used in QGIS. (Unfortunately, it seems that importing GDAL, a library made for working with geographical data, in this environment causes conflicts with cv2, which is why I highly recommend creating a new environment and linking it to an ipykernel to avoid this conflict.)

## Part 6: Quantifying the Seagrass Areas

*Input: Binary picture of the detected seagrass, pixel size  
Output: Calculated Seagrass Area*  
This section of the code calculates the detected seagrass area based on the pixel size, which is given through Section 2: Georeferencing. At a flying height of 100 metres, every pixel is approxiamtely equivalent to 5 cm. Based on the amount of detected pixels, the area is then calculated. 

In [24]:
# define the function that computes the total amount of white pixels and then calculates the area in cm2, m2, and ha based on that

def calculate_white_area(image, pixel_size):
 
    # Check if the image was loaded correctly
    if image is None:
        print("Error: Image not loaded correctly.")
        return

    # Count the number of white pixels (value 255)
    white_pixel_count = cv2.countNonZero(image[:,:,0])

    # Calculate the pixel area based on the pixel size
    area_per_pixel_in_cm2 = (pixel_size ** 2) * 10000
    area_per_pixel_in_m2 = (pixel_size ** 2)

    # Total area in square metres
    area_m2 = white_pixel_count * area_per_pixel_in_m2

    # Convert area to hectares
    area_hectares = area_m2 / 10000

    # Print the results
    print(f"Number of white pixels: {white_pixel_count}")
    print(f"Area covered by one pixels: {round(area_per_pixel_in_cm2, 2)} square centimetres")
    print(f"Covered area: {round(area_m2, 2)} square meters")
    print(f"Covered area: {round(area_hectares, 4)} hectares")

In [25]:
# Define inputs and run function
seagrass_image = cv2.imread(os.path.join(results_path, f"{image_name}_SeagrassBinary.tif"))

# Ground Sample Distance in centimeters / pixel
pixel_size = 0.05025239034331673

# Calculate and display the white area
calculate_white_area(seagrass_image, pixel_size)

Number of white pixels: 3544519
Area covered by one pixels: 25.25 square centimetres
Covered area: 8950.98 square meters
Covered area: 0.8951 hectares


Help on built-in function countNonZero:

countNonZero(...)
    countNonZero(src) -> retval
    .   @brief Counts non-zero array elements.
    .
    .   The function returns the number of non-zero elements in src :
    .   \f[\sum _{I: \; \texttt{src} (I) \ne0 } 1\f]
    .
    .   The function do not work with multi-channel arrays. If you need to count non-zero array
    .   elements across all the channels, use Mat::reshape first to reinterpret the array as
    .   single-channel. Or you may extract the particular channel using either extractImageCOI, or
    .   mixChannels, or split.
    .
    .   @note
    .   - If only whether there are non-zero elements is important, @ref hasNonZero is helpful.
    .   - If the location of non-zero array elements is important, @ref findNonZero is helpful.
    .   @param src single-channel array.
    .   @sa  mean, meanStdDev, norm, minMaxLoc, calcCovarMatrix
    .   @sa  findNonZero, hasNonZero



array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], shape=(6217, 3), dtype=uint8)

In [17]:
np.shape(seagrass_image)

(3568, 6217, 3)